In [3]:
import gensim
from tqdm import tqdm

### Wikipedia dump
- Download ```kawiki-latest-pages-articles-multistream.xml``` files from https://dumps.wikimedia.org/kawiki/latest/
- Parse text from xml files using WikiExtractor - https://github.com/attardi/wikiextractor
- Load ```kawiki.txt``` file

In [4]:
with open("../txt/kawiki.txt", "r", encoding="utf-8") as file:
    document = file.read()

document[:500]

' \nედუარდ შევარდნაძე\n\nედუარდ ამბროსის ძე შევარდნაძე (დ. 25 იანვარი, 1928, მამათი, დღევანდელი ლანჩხუთის მუნიციპალიტეტი — გ. 7 ივლისი, 2014, თბილისი) — ქართველი პოლიტიკოსი და სახელმწიფო მოღვაწე. 1985-1990 წლებში საბჭოთა კავშირის საგარეო საქმეთა მინისტრი, 1995–2003 წლებში საქართველოს პრეზიდენტი.\n\nსკკპ წევრი 1948 წლიდან. 1946 წლის ივლის-ოქტომბერში მუშაობდა საქართველოს ალკკ თბილისის ორჯონიკიძის რაიკომის ინსტრუქტორად. IX-XI მოწვევების უმაღლესი საბჭოს დეპუტატი. სოციალისტური შრომის გმირი (1981). სკკპ ცკ-'

### Remove Stopwords
- Use ```stopwords.txt``` file to clean up Georgian stopwords from the text

In [5]:
with open("../txt/stopwords.txt", "r", encoding="utf-8") as file:
    stopwords = [line.replace('\n','') for line in file.readlines()]
stopwords[:5]

['ა.შ.', 'აგერ', 'აგრეთვე', 'ალბათ', 'ამაზე']

In [102]:
import re
for s in tqdm(stopwords):
    document = re.sub(r"\b" + s + r"\b", '', document.replace('\n', ' '))

document[:500]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 373/373 [12:32<00:00,  2.02s/it]


'  ედუარდ შევარდნაძე  ედუარდ ამბროსის ძე შევარდნაძე (დ. 25 იანვარი, 1928, მამათი, დღევანდელი ლანჩხუთის მუნიციპალიტეტი — გ. 7 ივლისი, 2014, თბილისი) — ქართველი პოლიტიკოსი  სახელმწიფო მოღვაწე. 1985-1990 წლებში საბჭოთა კავშირის საგარეო საქმეთა მინისტრი, 1995–2003 წლებში საქართველოს პრეზიდენტი.  სკკპ წევრი 1948 წლიდან. 1946 წლის ივლის-ოქტომბერში მუშაობდა საქართველოს ალკკ თბილისის ორჯონიკიძის რაიკომის ინსტრუქტორად. IX-XI მოწვევების უმაღლესი საბჭოს დეპუტატი. სოციალისტური შრომის გმირი (1981). სკკპ ცკ-ის'

### Divide the document into sentences

In [114]:
sentences = re.split(r"(?<=\w\w\w)\.", re.sub(r'[^ა-ჰ0-9\.\-—–\s]',' ', document))
sentences[:5]

['  ედუარდ შევარდნაძე  ედუარდ ამბროსის ძე შევარდნაძე  დ. 25 იანვარი  1928  მამათი  დღევანდელი ლანჩხუთის მუნიციპალიტეტი — გ. 7 ივლისი  2014  თბილისი  — ქართველი პოლიტიკოსი  სახელმწიფო მოღვაწე',
 ' 1985-1990 წლებში საბჭოთა კავშირის საგარეო საქმეთა მინისტრი  1995–2003 წლებში საქართველოს პრეზიდენტი',
 '  სკკპ წევრი 1948 წლიდან',
 ' 1946 წლის ივლის-ოქტომბერში მუშაობდა საქართველოს ალკკ თბილისის ორჯონიკიძის რაიკომის ინსტრუქტორად',
 '   -   მოწვევების უმაღლესი საბჭოს დეპუტატი']

In [115]:
len(sentences)

1057879

In [119]:
def process_lines(sentences):
    for line in tqdm(sentences):
        yield gensim.utils.simple_preprocess(line, max_len=100)

In [120]:
tokens = list(process_lines(sentences))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1057879/1057879 [00:16<00:00, 64324.39it/s]


### Training
Common parameters to tune:
- min_count (int) – Ignores all words with total frequency lower than this.
- window (int) – The maximum distance between the current and predicted word within a sentence.
- size (int) – Dimensionality of the feature vectors.
- compute_loss (bool) – If True, computes and stores loss value which can be retrieved using model.get_latest_training_loss().
- workers (int) – Use these many worker threads to train the model (=faster training with multicore machines).

Defaults:
```class gensim.models.word2vec.Word2Vec(sentences=None, size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, negative=5, cbow_mean=1, hashfxn=<built-in function hash>, iter=5, null_word=0, trim_rule=None, sorted_vocab=1, batch_words=10000, compute_loss=False, callbacks=())```

I'll set ```min_count=3``` to filter typos and mine as deep as possible. ```workers=12``` works best for my CPU (8700K, 6-core, 12-thread).  

In [135]:
def accuracy(model):
    accuracy = model.wv.accuracy('../txt/questions-geography-ka.txt')
    sum_corr = len(accuracy[-1]['correct'])
    sum_incorr = len(accuracy[-1]['incorrect'])
    total = sum_corr + sum_incorr
    percent = lambda a: a / total * 100 if total != 0 else 0
    print('- TS: {}, Correct: {:.4f}%, Incorrect: {:.4f}%'.format(total, percent(sum_corr), percent(sum_incorr)))
    return percent(sum_corr)

In [123]:
from contextlib import contextmanager
import time
@contextmanager
def timeit(s):
    start = time.time()
    yield
    print(f"[{time.time()-start:3.0f}s] {s}\n\n")

In [142]:
with timeit(f" Done."):
    model = gensim.models.Word2Vec(tokens, size=300, min_count=2, window=20, workers=12)
    model.train(tokens, total_examples=len(tokens), epochs=10)
    accuracy(model)

- TS: 1265, Correct: 38.4190%, Incorrect: 61.5810%
[190s]  Done.




### Hyperparameter tuning

In [14]:
# Store results in global variable
results = []

In [19]:
d = 200
mc = 100
w = 20

dims = [64,100,128,150,200,256,300]
mincounts = [2, 4, 8, 16, 32, 64, 128]
windows = [2, 4, 6, 8]


for i,d in enumerate(dims):
    for j,mc in enumerate(mincounts):
        for k,w in enumerate(windows):
            with timeit(f"[{i+1}/{len(dims)}] [S:{d} MC:{mc} W:{w}] Model training complete."):
                model = gensim.models.Word2Vec(tokens, size=d, min_count=mc, window=w, workers=12, compute_loss=True)
                model.train(tokens, total_examples=len(tokens), epochs=10)
                acc = accuracy(model)
                print(f"- Vocab size:{len(model.wv.vocab)}\n- Compute Loss: {model.get_latest_training_loss()}")
                results.append({
                    'size':d,
                    'min_count':mc,
                    'window':w,
                    'vocab':len(model.wv.vocab),
                    'accuracy':acc,
                    'loss':model.get_latest_training_loss()
                })

- TS: 4595, Correct: 1.6104%, Incorrect: 98.3896%
- Vocab size:15247
- Compute Loss: 0.0
[ 54s] [1/6] [S:10 MC:100 W:20] Model training complete.




KeyboardInterrupt: 

### Model testing

In [21]:
w1 = ['საფერავი']
model.wv.most_similar (positive=w1)

[('რქაწითელი', 0.9220234155654907),
 ('ქინძმარაული', 0.8812519311904907),
 ('ციცქა', 0.8682762384414673),
 ('საადრეო', 0.8441500663757324),
 ('კრახუნა', 0.8396971821784973),
 ('ცოლიკოური', 0.8338011503219604),
 ('ჩხავერი', 0.823092520236969),
 ('გორული', 0.8198326230049133),
 ('ჯიშებიდან', 0.8062148690223694),
 ('ატენური', 0.7817213535308838)]

In [22]:
def analogie(a, b, c, model):
    return model.wv.most_similar(positive=[b,c], negative=[a], topn=10)

In [23]:
analogie("თბილისი","საქართველო","კიევი", model)

[('რუსეთი', 0.5468760132789612),
 ('პოლტავა', 0.5239676833152771),
 ('უკრაინა', 0.5236309170722961),
 ('მოლდოვა', 0.5170091390609741),
 ('ოსმალეთი', 0.5034421682357788),
 ('ვოლინსკის', 0.500557541847229),
 ('სერბეთი', 0.4980471134185791),
 ('ჩერნიგოვი', 0.49603569507598877),
 ('ირანი', 0.49343639612197876),
 ('ყირიმი', 0.4855012893676758)]

In [150]:
model.predict_output_word(['ჩემი','ხატია','სამშობლო','სახატე','მთელი'])

[('ჩემი', 0.9271839),
 ('ჩემს', 0.025721798),
 ('ვარ', 0.0060025216),
 ('შენი', 0.005886703),
 ('შენ', 0.0055993847),
 ('სამშობლო', 0.0051750545),
 ('მე', 0.004013303),
 ('ჩემო', 0.001358541),
 ('ვართ', 0.00045134505),
 ('შენს', 0.00038922738)]

### Persist model
- and export as .tsv for http://projector.tensorflow.org/

In [145]:
with timeit("Model saved"):
    model.save("../model/kawiki_1250MB")

[ 11s] Model saved




In [129]:
def word2vec2tensor(model, tensor_filename, binary=False):
    outfiletsv = tensor_filename + '_tensor.tsv'
    outfiletsvmeta = tensor_filename + '_metadata.tsv'

    with open(outfiletsv, 'w+', encoding='utf-8') as file_vector:
        with open(outfiletsvmeta, 'w+', encoding='utf-8') as file_metadata:
            file_metadata.write('word\tcount\n')
            for word in tqdm(model.wv.index2word):
                wordstring = gensim.utils.to_utf8(word).decode("utf-8")
                countstring = str(model.wv.vocab[word].count)
                file_metadata.write(wordstring + '\t' + countstring + '\n')
                vector_row = '\t'.join(str(x) for x in model.wv[word])
                file_vector.write(vector_row + '\n')

In [144]:
word2vec2tensor(model,"../tsv/kawiki_1250MB")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 356756/356756 [00:53<00:00, 6651.86it/s]
